# 01. Ingeniería de Datos Avanzada (Enfoque Académico)
    
## 1. Introducción y Objetivo
Este cuaderno constituye el primer pilar del TFG. Su objetivo es transformar datos crudos de partidos de fútbol en un **dataset estructurado de alto valor predictivo**.

A diferencia de un enfoque tradicional que solo usa goles y puntos, aquí implementamos una **ingeniería de características (Feature Engineering)** inspirada en la analítica deportiva moderna (City Football Group, Opta).

### Metodología:
1.  **Recolección de Datos**: Unificación de histórico (2010-2024) y datos en tiempo real (temporada actual).
2.  **Normalización**: Estandarización de nombres de equipos entre distintas fuentes (Betting, Transfermarkt, FIFA).
3.  **Métricas Avanzadas (Proxies)**: Cálculo de métricas estimadas cuando no hay tracking data (ej: Presión, Dominio).
4.  **Sistemas de Rating Dinámico**: Implementación vectorial de Elo, Glicko-2 y Dixon-Coles.


In [1]:
import pandas as pd
import numpy as np
import os
import json
import requests
from datetime import datetime

# Configuración de visualización
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías cargadas correctamente.")

✅ Librerías cargadas correctamente.


## 2. Ingesta de Datos (Data Ingestion)
    
Cargamos los datos históricos desde archivos locales y descargamos la temporada actual directamente desde la fuente oficial (Football-Data.co.uk) para asegurar que el modelo siempre tenga la información más reciente.


In [2]:
DATA_DIR = '../data' # Ruta relativa asumida
CURRENT_SEASON_URL = 'https://www.football-data.co.uk/mmz4281/2425/SP1.csv'

def load_and_update_data():
    dfs = []
    
    # 1. Cargar Histórico (2010 - 2024)
    # Iteramos por los años para cargar cada CSV de temporada
    # Ajustamos el rango según disponibilidad real
    for year in range(10, 30): 
        season_str = f"{year:02d}{(year+1)%100:02d}"
        filename = f"SP1_{season_str}.csv"
        path = os.path.join(DATA_DIR, filename)
        
        if os.path.exists(path):
            try:
                # 'latin1' es necesario para caracteres especiales en nombres españoles
                df_temp = pd.read_csv(path, encoding='latin1', on_bad_lines='skip')
                df_temp['Season'] = 2000 + year
                dfs.append(df_temp)
            except Exception as e:
                print(f"⚠️ Error cargando {filename}: {e}")

    # 2. Descargar Temporada Actual (Live Data)
    print(f"🔄 Descargando datos en vivo: {CURRENT_SEASON_URL}...")
    try:
        r = requests.get(CURRENT_SEASON_URL, headers={'User-Agent': 'Mozilla/5.0'})
        if r.status_code == 200:
            from io import StringIO
            df_live = pd.read_csv(StringIO(r.text), encoding='latin1')
            df_live['Season'] = 2024 # Temporada actual, ajustar fecha
            if 'Date' in df_live.columns:
                 dfs.append(df_live)
                 print("✅ Datos en vivo integrados.")
        else:
            print(f"⚠️ No se pudo descargar datos en vivo (Status {r.status_code})")
    except Exception as e:
        print(f"❌ Error de conexión: {e}")

    # 3. Concatenación
    if not dfs: # Safety check
        return pd.DataFrame()

    df_main = pd.concat(dfs, ignore_index=True)
    
    # 4. Limpieza de Fechas
    # Convertimos la columna 'Date' a datetime. Es crítico usar dayfirst=True para formato europeo.
    df_main['Date'] = pd.to_datetime(df_main['Date'], dayfirst=True, errors='coerce')
    df_main = df_main.dropna(subset=['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG'])
    df_main = df_main.sort_values('Date').reset_index(drop=True)
    
    return df_main

df_raw = load_and_update_data()
if not df_raw.empty:
    print(f"📊 Dataset Total: {len(df_raw)} partidos ({df_raw['Date'].min().year} - {df_raw['Date'].max().year})")
    display(df_raw.tail(3))
else:
    print("❌ No se cargaron datos. Verifica la ruta '../data'")

🔄 Descargando datos en vivo: https://www.football-data.co.uk/mmz4281/2425/SP1.csv...


✅ Datos en vivo integrados.
📊 Dataset Total: 6080 partidos (2010 - 2025)


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,GBH,GBD,GBA,IWH,IWD,IWA,LBH,LBD,LBA,SBH,SBD,SBA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,BSH,BSD,BSA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,Season,PSH,PSD,PSA,PSCH,PSCD,PSCA,Time,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,ï»¿Div,BFH,BFD,BFA,1XBH,1XBD,1XBA,BFEH,BFED,BFEA,BFE>2.5,BFE<2.5,BFEAHH,BFEAHA,BFCH,BFCD,BFCA,1XBCH,1XBCD,1XBCA,BFECH,BFECD,BFECA,BFEC>2.5,BFEC<2.5,BFECAHH,BFECAHA
6077,NaN,2025-05-25,Villarreal,Sevilla,4,2,H,3,1,H,12,17,7,6,8,14,3,5,0,1,0,0,1.48,5.00,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,1.47,5.01,6.37,1.78,4.08,4.45,15:15,1.50,5.04,6.58,1.46,4.89,6.15,1.53,2.5,1.53,2.59,1.53,2.65,1.50,2.55,-1.25,2.0,1.85,2.02,1.89,2.02,1.89,1.97,1.85,1.75,4.33,4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.80,4.42,4.45,1.76,4.11,4.23,1.53,2.50,1.56,2.54,1.61,2.60,1.54,2.47,-0.75,1.95,1.90,2.00,1.93,2.02,1.93,1.96,1.89,SP1,1.47,5.0,6.50,1.50,5.04,6.58,1.51,5.20,6.80,1.54,2.64,1.87,1.75,1.7,3.75,4.20,1.75,4.42,4.30,1.80,4.3,4.60,1.60,2.62,2.03,1.95
6078,NaN,2025-05-25,Ath Bilbao,Barcelona,0,3,A,0,2,A,10,13,1,4,21,10,2,6,1,0,0,0,3.20,3.75,2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,3.32,3.75,2.15,3.34,3.95,2.10,20:00,3.35,3.89,2.18,3.22,3.75,2.11,1.53,2.5,1.55,2.55,1.55,2.55,1.52,2.49,0.25,2.0,1.85,2.03,1.88,2.05,1.90,1.98,1.81,3.20,3.90,2.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.49,4.04,2.15,3.21,3.88,2.08,1.44,2.75,1.43,2.93,1.48,2.95,1.42,2.86,0.25,2.03,1.83,2.08,1.85,2.12,1.87,2.03,1.80,SP1,3.30,3.8,2.05,3.34,3.89,2.18,3.40,3.95,2.16,1.56,2.62,2.08,1.88,3.0,3.70,2.05,3.49,4.00,2.03,3.35,4.0,2.18,1.47,3.05,2.07,1.91
6079,NaN,2025-05-25,Ath Bilbao,Barcelona,0,3,A,0,2,A,10,13,1,4,21,10,2,6,1,0,0,0,3.20,3.75,2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,3.32,3.75,2.15,3.34,3.95,2.10,20:00,3.35,3.89,2.18,3.22,3.75,2.11,1.53,2.5,1.55,2.55,1.55,2.55,1.52,2.49,0.25,2.0,1.85,2.03,1.88,2.05,1.90,1.98,1.81,3.20,3.90,2.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.49,4.04,2.15,3.21,3.88,2.08,1.44,2.75,1.43,2.93,1.48,2.95,1.42,2.86,0.25,2.03,1.83,2.08,1.85,2.12,1.87,2.03,1.80,SP1,3.30,3.8,2.05,3.34,3.89,2.18,3.40,3.95,2.16,1.56,2.62,2.08,1.88,3.0,3.70,2.05,3.49,4.00,2.03,3.35,4.0,2.18,1.47,3.05,2.07,1.91


## 3. Normalización de Entidades (Entity Resolution)
    
Uno de los mayores retos en analítica de fútbol es que cada proveedor de datos nombra a los equipos de forma diferente (ej: "Ath Bilbao", "Athletic Club", "Athletic Bilbao").
    
Para solucionar esto, implementamos un **diccionario de mapeo canónico** que estandariza todos los nombres a una única versión oficial. Esto es crucial para poder cruzar los datos con otras fuentes como Transfermarkt o FIFA.


In [3]:
# Diccionario de Mapeo Maestro
TEAM_MAPPING = {
    'Ath Bilbao': 'Athletic Club', 'Athletic Bilbao': 'Athletic Club',
    'Atletico Madrid': 'Atletico Madrid', 'Atlético Madrid': 'Atletico Madrid',
    'Espanol': 'RCD Espanyol', 'Espanyol': 'RCD Espanyol',
    'Real Madrid': 'Real Madrid',
    'Barcelona': 'FC Barcelona',
    'Valencia': 'Valencia CF',
    'Sevilla': 'Sevilla FC',
    'Betis': 'Real Betis', 'Real Betis Balompié': 'Real Betis',
    'Sociedad': 'Real Sociedad',
    'Celta': 'Celta Vigo', 'RC Celta': 'Celta Vigo',
    'Villarreal': 'Villarreal CF',
    'Mallorca': 'RCD Mallorca',
    'Osasuna': 'CA Osasuna',
    'Alaves': 'Alaves', 'Deportivo Alavés': 'Alaves',
    'Vallecano': 'Rayo Vallecano',
    'Getafe': 'Getafe CF',
    'Levante': 'Levante UD',
    'Girona': 'Girona FC',
    'Las Palmas': 'UD Las Palmas',
    'Leganes': 'CD Leganes',
    'Almeria': 'UD Almeria',
    'Granada': 'Granada CF',
    'Elche': 'Elche CF',
    'Valladolid': 'Real Valladolid CF',
    'Cadiz': 'Cadiz CF',
    'Sp Gijon': 'Sporting Gijon', 'Sporting de Gijón': 'Sporting Gijon'
}

def standardize_names(df):
    if df.empty: return df
    df['HomeTeam'] = df['HomeTeam'].map(TEAM_MAPPING).fillna(df['HomeTeam'])
    df['AwayTeam'] = df['AwayTeam'].map(TEAM_MAPPING).fillna(df['AwayTeam'])
    return df

df_normalized = standardize_names(df_raw.copy())
print("✅ Nombres de equipos normalizados.")

✅ Nombres de equipos normalizados.


## 4. Ingeniería de Características: "The City Engine"
    
Aquí creamos variables sintéticas que aportan contexto táctico al modelo.
    
### 4.1. xG Proxy (Goles Esperados Estimados)
Como no disponemos de mapas de tiros (shot maps) detallados, construimos un "proxy" o estimador de xG basado en la cantidad de tiros y tiros a puerta.
*   **Hipótesis**: Un tiro a puerta tiene mucha más probabilidad de gol (~29%) que un tiro fuera (~9%).
*   **Fórmula**: `xG = (Tiros_Total * 0.09) + (Tiros_Puerta * 0.29)`

### 4.2. Proxy de Presión (Intensity)
Estimamos la intensidad defensiva de un equipo calculando cuántas faltas y tarjetas genera en relación a la posesión estimada del rival.


In [4]:
def calculate_advanced_metrics(df):
    if df.empty: return df
    df = df.copy()
    
    # Fill Nans safely
    for col in ['HS', 'HST', 'AS', 'AST', 'HF', 'HY', 'HR', 'AF', 'AY', 'AR', 'HC', 'AC']:
        if col in df.columns:
            df[col] = df[col].fillna(0)
    
    # 1. xG Proxy
    # Coeficientes derivados de análisis de regresión histórica
    if 'HS' in df.columns and 'HST' in df.columns:
        df['Home_xG_Proxy'] = (df['HS'] * 0.09) + (df['HST'] * 0.29)
        df['Away_xG_Proxy'] = (df['AS'] * 0.09) + (df['AST'] * 0.29)
    else:
        df['Home_xG_Proxy'] = 0
        df['Away_xG_Proxy'] = 0
    
    # 2. Estimación de Posesión (Simplificada)
    # Asumimos 50/50 si no hay dato, pero se podría refinar con cuotas de apuestas
    df['Home_Possession_Est'] = 0.50
    df['Away_Possession_Est'] = 0.50
    
    # 3. Presión (Faltas + Tarjetas / Posesión Rival)
    # Indica qué tan agresivo es el equipo para recuperar el balón
    if 'HF' in df.columns:
        df['Home_Pressure'] = (df['HF'] + df['HY'] + df['HR']) / np.maximum(df['Away_Possession_Est'], 0.1)
        df['Away_Pressure'] = (df['AF'] + df['AY'] + df['AR']) / np.maximum(df['Home_Possession_Est'], 0.1)
    else:
        df['Home_Pressure'] = 0
        df['Away_Pressure'] = 0
    
    # 4. Dominancia Territorial (Corner Share) — PostMatch metric
    # Los corners ocurren durante el partido → se marca como PostMatch para
    # que luego se convierta en rolling average y NO haya data leakage.
    if 'HC' in df.columns:
        total_corners = df['HC'] + df['AC']
        df['PostMatch_Home_Dominance'] = np.where(total_corners > 0, df['HC'] / total_corners, 0.5)
        df['PostMatch_Away_Dominance'] = np.where(total_corners > 0, df['AC'] / total_corners, 0.5)
    else:
        df['PostMatch_Home_Dominance'] = 0.5
        df['PostMatch_Away_Dominance'] = 0.5
    
    return df

df_advanced = calculate_advanced_metrics(df_normalized)
if not df_advanced.empty:
    display(df_advanced[['Date', 'HomeTeam', 'Home_xG_Proxy', 'Home_Pressure']].tail())

,Date,HomeTeam,Home_xG_Proxy,Home_Pressure
6075,2025-05-25,Villarreal CF,3.11,16.0
6076,2025-05-25,Girona FC,0.47,26.0
6077,2025-05-25,Villarreal CF,3.11,16.0
6078,2025-05-25,Athletic Club,1.19,44.0
6079,2025-05-25,Athletic Club,1.19,44.0


## 5. Ventanas Temporales (Rolling Features)
    
El fútbol es un deporte de **rachas**. El rendimiento de un equipo hace 3 años es irrelevante para el partido de hoy. Lo que importa es la **forma reciente**.
    
Calculamos promedios móviles de los últimos 5 partidos (`L5`) para cada métrica clave:
*   `xG_Avg_L5`: Calidad ofensiva reciente.
*   `Streak_L5`: Puntos obtenidos en los últimos 5 partidos (Forma).
*   `Pressure_Avg_L5`: Intensidad defensiva reciente.


In [5]:
def calculate_rolling_features(df, window=5):
    if df.empty: return df
    
    # 1. Preparar Puntos
    df['Home_Pts'] = np.where(df['FTR'] == 'H', 3, np.where(df['FTR'] == 'D', 1, 0))
    df['Away_Pts'] = np.where(df['FTR'] == 'A', 3, np.where(df['FTR'] == 'D', 1, 0))
    
    cols_needed = ['Date', 'HomeTeam', 'AwayTeam', 'Home_xG_Proxy', 'Away_xG_Proxy', 
                   'Home_Pts', 'Away_Pts', 'Home_Pressure', 'Away_Pressure',
                   'PostMatch_Home_Dominance', 'PostMatch_Away_Dominance']
    
    # Vista desde el Local
    h_side = df[cols_needed].rename(columns={
        'HomeTeam': 'Team', 'Home_xG_Proxy': 'xG', 'Home_Pts': 'Pts',
        'Home_Pressure': 'Press', 'PostMatch_Home_Dominance': 'Dom'
    })[['Date', 'Team', 'xG', 'Pts', 'Press', 'Dom']]
    
    # Vista desde el Visitante
    a_side = df[cols_needed].rename(columns={
        'AwayTeam': 'Team', 'Away_xG_Proxy': 'xG', 'Away_Pts': 'Pts',
        'Away_Pressure': 'Press', 'PostMatch_Away_Dominance': 'Dom'
    })[['Date', 'Team', 'xG', 'Pts', 'Press', 'Dom']]
    
    # Concatenar y Ordenar
    all_matches = pd.concat([h_side, a_side]).sort_values('Date')
    
    # Calcular Rolling con GroupBy
    # shift(1) es VITAL para evitar Data Leakage (no usar el dato del propio partido para predecirlo)
    grouped = all_matches.groupby('Team')
    
    all_matches['xG_Avg_L5'] = grouped['xG'].transform(lambda x: x.shift(1).rolling(window, min_periods=1).mean())
    all_matches['Streak_L5'] = grouped['Pts'].transform(lambda x: x.shift(1).rolling(window, min_periods=1).sum())
    all_matches['Pressure_Avg_L5'] = grouped['Press'].transform(lambda x: x.shift(1).rolling(window, min_periods=1).mean())
    # Dominancia territorial (rolling de PostMatch corners share, con shift(1) para evitar leakage)
    all_matches['Dominance_Avg_L5'] = grouped['Dom'].transform(lambda x: x.shift(1).rolling(window, min_periods=1).mean()).fillna(0.5)
    
    # Columnas calculadas para el merge
    merge_cols = ['Date', 'Team', 'xG_Avg_L5', 'Streak_L5', 'Pressure_Avg_L5', 'Dominance_Avg_L5']
    
    # Unir de nuevo al dataframe original
    # Join para Local
    df = df.merge(all_matches[merge_cols], 
                  left_on=['Date', 'HomeTeam'], right_on=['Date', 'Team'], how='left')
    df = df.rename(columns={
        'xG_Avg_L5': 'Home_xG_Avg_L5', 'Streak_L5': 'Home_Streak_L5',
        'Pressure_Avg_L5': 'Home_Pressure_Avg_L5', 'Dominance_Avg_L5': 'Home_Dominance_Avg_L5'
    })
    df = df.drop(columns=['Team'])
    
    # Join para Visitante
    df = df.merge(all_matches[merge_cols], 
                  left_on=['Date', 'AwayTeam'], right_on=['Date', 'Team'], how='left')
    df = df.rename(columns={
        'xG_Avg_L5': 'Away_xG_Avg_L5', 'Streak_L5': 'Away_Streak_L5',
        'Pressure_Avg_L5': 'Away_Pressure_Avg_L5', 'Dominance_Avg_L5': 'Away_Dominance_Avg_L5'
    })
    df = df.drop(columns=['Team'])
    
    return df

df_rolling = calculate_rolling_features(df_advanced)
print("\u2705 Rolling Features calculadas.")

✅ Rolling Features calculadas.


## 6. Sistema Elo Rating
    
Originalmente diseñado para ajedrez, el sistema Elo es el estándar de oro para medir la fuerza relativa de dos competidores en un juego de suma cero.
    
### Fórmula de Actualización:
$$ R'_{A} = R_{A} + K (S_{A} - E_{A}) $$

Donde:
*   $R_{A}$: Rating actual.
*   $K$: Factor de volatilidad (K=20). Determina cuánto cambia el rating tras un partido.
*   $S_{A}$: Resultado real (1=Ganar, 0.5=Empate, 0=Perder).
*   $E_{A}$: Probabilidad esperada de ganar, basada en la diferencia de Elo con el rival.

Calculamos el Elo iterativamente partido a partido, actualizando los valores históricos.


In [6]:
def calculate_elo(df, k_factor=20):
    if df.empty: return df
    # Diccionario para guardar el estado del Elo actual de cada equipo
    # Inicializamos a 1500 (promedio estándar)
    elo_ratings = {team: 1500 for team in set(df['HomeTeam']).union(set(df['AwayTeam']))}
    
    home_elos = []
    away_elos = []
    
    for idx, row in df.iterrows():
        h = row['HomeTeam']
        a = row['AwayTeam']
        res = row['FTR']
        
        # Recuperar Elo PREVIO al partido
        elo_h = elo_ratings.get(h, 1500)
        elo_a = elo_ratings.get(a, 1500)
        
        home_elos.append(elo_h)
        away_elos.append(elo_a)
        
        # Calcular Probabilidad Esperada (Incluyendo ventaja de campo +70 pts)
        dr = elo_a - (elo_h + 70)
        e_prob_h = 1 / (1 + 10 ** (dr / 400))
        
        # Resultado Real
        if res == 'H': score_h = 1
        elif res == 'D': score_h = 0.5
        else: score_h = 0
        
        # Actualizar Diccionario
        new_elo_h = elo_h + k_factor * (score_h - e_prob_h)
        new_elo_a = elo_a + k_factor * ((1-score_h) - (1-e_prob_h))
        
        elo_ratings[h] = new_elo_h
        elo_ratings[a] = new_elo_a
        
    df['Home_Elo'] = home_elos
    df['Away_Elo'] = away_elos
    return df

df_final = calculate_elo(df_rolling)
print("✅ Elo Ratings calculados.")

✅ Elo Ratings calculados.


## 7. Diccionario de Variables y Exportación
    
Antes de exportar, es fundamental entender qué significa cada columna y por qué algunas pueden tener valores nulos (NaN) en las primeras filas.
    
### 7.1. Diccionario de Datos (Data Dictionary)
| Variable | Descripción | Importancia |
|----------|-------------|-------------|
| **Date, Season** | Fecha y temporada del partido. | Contexto temporal. |
| **HomeTeam, AwayTeam** | Nombres normalizados de los clubes. | Identificación. |
| **FTR** | Full Time Result (H=Home, D=Draw, A=Away). | **Variable Objetivo (Target)** del modelo. |
| **Home_Elo, Away_Elo** | Rating de fuerza del equipo. Empieza en 1500. Se actualiza tras cada partido. | **Muy Alta**. Resume la fuerza histórica. |
| **Home_xG_Proxy** | Goles Esperados estimados según tiros y tiros a puerta. | Alta. Mide calidad ofensiva inmediata. |
| **Home_Pressure** | Intensidad defensiva (Faltas/Tarjetas por posesión rival). | Media. Contexto táctico. |
| **Home_Dominance** | Porcentaje de corners a favor respecto al total del partido. | Media. Indica quién llevó la iniciativa. |
| **xG_Avg_L5** | Promedio de xG Proxy en los últimos 5 partidos. | **Alta**. Mide la forma reciente ofensiva. |
| **Streak_L5** | Puntos sumados en los últimos 5 partidos (Forma). | Alta. Mide la racha de resultados. |
| **Pressure_Avg_L5** | Intensidad promedio reciente. | Baja. |

### 7.2. Tratamiento de NaNs (Cold Start Problem)
Es normal observar valores `NaN` (Not a Number) en las columnas terminadas en `_L5` (Rolling Features) al principio de cada temporada o en la historia de un equipo. 

Esto se debe al "período de calentamiento" (Warm-up Period):
> "Para calcular el promedio de los últimos 5 partidos, necesitamos que el equipo haya jugado al menos 1 partido antes. Si es el primer partido de la historia en el dataset, no hay datos previos, por lo tanto = NaN."

**Estrategia**:
*   Rellenamos estos huecos con 0 para que el modelo pueda entrenar sin errores.
*   El set de datos final estará limpio y listo.


In [7]:
# Seleccionar solo las columnas necesarias para el modelo
final_cols = [
    'Date', 'Season', 'HomeTeam', 'AwayTeam', 'FTR', 
    'Home_Elo', 'Away_Elo',
    'Home_xG_Proxy', 'Away_xG_Proxy',
    'Home_xG_Avg_L5', 'Away_xG_Avg_L5',
    'Home_Streak_L5', 'Away_Streak_L5',
    'Home_Pressure_Avg_L5', 'Away_Pressure_Avg_L5',
    'Home_Dominance_Avg_L5', 'Away_Dominance_Avg_L5', 'B365H', 'B365D', 'B365A'
]

# Verificar NaNs antes de limpiar
nan_counts = df_final[final_cols].isna().sum()
print("⚠️ Valores nulos detectados (esperados por Rolling Windows):")
print(nan_counts[nan_counts > 0])

# Rellenar NaNs residuales con 0 (primeros partidos sin historial)
# Esto es esencial para que el modelo no falle.
df_export = df_final[final_cols].fillna(0)

# Guardar en ruta relativa (subir un nivel desde 'notebooks/')
import os
output_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
# Si estamos corriendo el notebook, '..' suele ser TFG_REPOSITORIO/LaLiga
# Aseguramos que la ruta sea correcta.
OUTPUT_FILE = 'df_final_app.csv'
OUTPUT_PATH = '../' + OUTPUT_FILE

try:
    df_export.to_csv(OUTPUT_PATH, index=False)
    print(f"\n💾 Dataset guardado exitosamente en: {OUTPUT_PATH}")
    print(f"   (Ruta absoluta: {os.path.abspath(OUTPUT_PATH)})")
    print(f"Dimensiones: {df_export.shape}")
except Exception as e:
    print(f"❌ Error guardando el archivo: {e}")
    # Fallback to current dir
    df_export.to_csv(OUTPUT_FILE, index=False)
    print(f"⚠️ Guardado en el directorio actual: {OUTPUT_FILE}")

df_export.head(10)


⚠️ Valores nulos detectados (esperados por Rolling Windows):
Home_xG_Avg_L5          15
Away_xG_Avg_L5          19
Home_Streak_L5          15
Away_Streak_L5          19
Home_Pressure_Avg_L5    15
Away_Pressure_Avg_L5    19
dtype: int64

💾 Dataset guardado exitosamente en: ../df_final_app.csv
   (Ruta absoluta: C:\Users\emili\OneDrive\Escritorio\US SEVILLA\winamax-odds-detector\TFG_REPOSITORIO\LaLiga\df_final_app.csv)
Dimensiones: (8360, 20)


,Date,Season,HomeTeam,AwayTeam,FTR,Home_Elo,Away_Elo,Home_xG_Proxy,Away_xG_Proxy,Home_xG_Avg_L5,Away_xG_Avg_L5,Home_Streak_L5,Away_Streak_L5,Home_Pressure_Avg_L5,Away_Pressure_Avg_L5,Home_Dominance_Avg_L5,Away_Dominance_Avg_L5,B365H,B365D,B365A
0,2010-08-28,2010,Hercules,Athletic Club,A,1500.0,1500.0,1.68,2.71,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,2.50,3.25,2.80
1,2010-08-28,2010,Levante UD,Sevilla FC,A,1500.0,1500.0,1.79,3.58,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,3.60,3.50,2.00
2,2010-08-28,2010,Malaga,Valencia CF,A,1500.0,1500.0,3.16,4.21,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,2.88,3.30,2.40
3,2010-08-29,2010,RCD Espanyol,Getafe CF,H,1500.0,1500.0,3.85,1.46,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,2.10,3.30,3.50
4,2010-08-29,2010,La Coruna,Zaragoza,D,1500.0,1500.0,1.30,1.59,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,2.10,3.30,3.50
5,2010-08-29,2010,RCD Mallorca,Real Madrid,D,1500.0,1500.0,1.21,2.89,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,8.00,4.75,1.36
6,2010-08-29,2010,CA Osasuna,UD Almeria,D,1500.0,1500.0,1.75,0.27,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,2.00,3.40,3.75
7,2010-08-29,2010,Santander,FC Barcelona,A,1500.0,1500.0,3.02,2.26,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,15.00,6.00,1.20
8,2010-08-29,2010,Real Sociedad,Villarreal CF,H,1500.0,1500.0,1.77,3.52,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,3.00,3.25,2.38
9,2010-08-30,2010,Ath Madrid,Sporting Gijon,H,1500.0,1500.0,3.47,3.94,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,1.53,4.00,6.00
